In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Base de données - statistiques des jeux de league of legends dans les 10 premier minutes (un jeu peut durer 40 min en moyenne)
# ligue diamant (à peu près le 1% des meilleurs joueurs ) 

lolgames = pd.read_csv("/kaggle/input/league-of-legends-diamond-ranked-games-10-min/high_diamond_ranked_10min.csv")


In [ ]:
#le jeu a quatre objetifs uniques independantes dans les 10 premiers min : premier meurtre, premier dragon, premier herald, tours détruites
#le but est predire la victoire du equipe bleu

predicteurs=lolgames[['blueFirstBlood','blueDragons', 'blueHeralds','blueTowersDestroyed']]
cible=lolgames.blueWins

In [ ]:
import matplotlib.pylab as plt
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import sklearn.metrics

In [ ]:
# 30%  c'est un echantillon d'entrainement
pred_train, pred_test, cib_train, cib_test = train_test_split(predicteurs, cible, test_size=.3)

In [ ]:
#Profondeur initiale 
classifier=DecisionTreeClassifier(max_depth=2)
classifier=classifier.fit(pred_train,cib_train)

In [ ]:
predictions=classifier.predict(pred_test)
predictions_train=classifier.predict(pred_train)

In [ ]:
#mesurer la precision pour savoire s'il y a overfitting

from sklearn.metrics import accuracy_score
print('Precision sur train:', accuracy_score(cib_train,predictions_train))
print('Precision sur test:', accuracy_score(cib_test, predictions))

In [ ]:
from sklearn import tree
from io import StringIO
from IPython.display import Image

In [ ]:
attributes=['blueFirstBlood','blueDragons', 'blueHeralds','blueTowersDestroyed']

In [ ]:
out=StringIO()
tree.export_graphviz(classifier, out_file='arbre_lol.dot',feature_names=attributes)

In [ ]:
!dot -Tpng arbre_lol.dot -o arbre_lol.png -Gdpi=600

In [ ]:
Image(filename='arbre_lol.png')

#figure - 63% de probabilité de victoire si l'equipe prend le dragon dans les 10 premier minutes, et baisse jusqu'a 41% sinon
#70% de probabilité si l'equipe prend la premiere meurtre et le premier dragon qui decend jusqu'a 54% avec seulement le dragon
#51% de probabilité si l'equipe prend la premiere meurtre mais non le premier dragon qui decend jusqu'a 33% sans aucune des deux


In [ ]:
#Reglage du hyper-paramtre profondeur avec graphique du precision vs profondeur

from matplotlib.pyplot import plot
max_depth_range = list(range(1, 6))
accuracy = [] 
for depth in max_depth_range:
    
    clf = DecisionTreeClassifier(max_depth = depth, random_state = 0)
    clf.fit(pred_train, cib_train)    
    score = clf.score(pred_test, cib_test)
    accuracy.append(score)
    
plot(max_depth_range, accuracy)
plt.xlabel("precision")
plt.ylabel("profondeur")

In [ ]:
#nouvelle profondeur
clf_limite = DecisionTreeClassifier(max_depth=4, random_state=0)
clf_limite = clf_limite.fit(pred_train,cib_train)

In [ ]:
from sklearn import tree
from io import StringIO
from IPython.display import Image
out=StringIO()
tree.export_graphviz(clf_limite, out_file='arbre_lol.dot',feature_names=attributes)
!dot -Tpng arbre_lol.dot -o arbre_lol.png -Gdpi=600
Image(filename='arbre_lol.png')
#meme logique d'interpretation qu'avant avec de la premiere tour et le herald
#ex: sans dragon, sans premiere meurtre, sans tours detruites, sans herald - 32% probabilité de victoire
#ex: sans dragon, sans premiere meurtre, sans tours detruites, avec herald - 40% probabilité de victoire

In [ ]:
#importance des objetifs dans la victoire 54% premiere dragon, 33% premiere meurtre, 7% les tours detruites, 4% le premier herald

importances = pd.DataFrame({'variable':pred_train.columns,'importance':np.round(clf_limite.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False)
print(importances)